# 표본 추출

## 필요 라이브러리 import

In [2]:
import pandas as pd
import numpy as np

## 데이터셋 생성

In [5]:
X_value = np.arange(40).reshape(20, 2)
y_value = np.arange(20)

sample_df = pd.DataFrame(np.column_stack((X_value, y_value)), columns=['X_1', 'X_2', 'result'])

# 생성된 데이터 확인
print(sample_df.shape)

(20, 3)


## 표본추출
### 단순확률표본추출
- python pandas 모듈의 DataFrame.sample() 메소드 이용

In [7]:
# 5개의 랜덤표본추출
sample_df.sample(n=5, random_state=1001)

,X_1,X_2,result
1,2,3,1
15,30,31,15
0,0,1,0
2,4,5,2
18,36,37,18


In [8]:
# 특정 비율만큼 추출
sample_df.sample(frac=0.5, random_state=1001)

,X_1,X_2,result
1,2,3,1
15,30,31,15
0,0,1,0
2,4,5,2
18,36,37,18
7,14,15,7
10,20,21,10
6,12,13,6
19,38,39,19
4,8,9,4


In [10]:
# 복원 무작위 표본 추출
rep_df = sample_df.sample(frac=0.2, random_state=1001)
rep_df

,X_1,X_2,result
1,2,3,1
15,30,31,15
0,0,1,0
2,4,5,2


In [11]:
# replace = True : 복원 추출
# rep_df 4개 샘플을 복원추출로 10개로 만들기
rep_df.sample(n=10, random_state=1001, replace=True)

,X_1,X_2,result
15,30,31,15
15,30,31,15
15,30,31,15
0,0,1,0
0,0,1,0
2,4,5,2
1,2,3,1
1,2,3,1
0,0,1,0
2,4,5,2


In [15]:
## Dataframe 내의 특정 컬럼의 값을 기준으로 가중치를 부여하여 무작위 표본 추출
# weights : 가중치 반영할 필드값
sample_df.sample(n=5, weights='result')

,X_1,X_2,result
13,26,27,13
2,4,5,2
14,28,29,14
7,14,15,7
12,24,25,12


### 계통표본추출
- sysmetic_sampling 함수 정의. 필요 샘플 수를 입력받아 간격을 구하여 샘플 추출

In [16]:
# 계통표본추출 함수 정의
def sysmetic_sampling(data, n):
    count = len(data)   # 모집단 수
    sample_count = count // n
    index = data[:sample_count].sample(1).index
    intoin = index - 0   # 샘플 간 간격
    sys_df = pd.DataFrame()
    while len(sys_df) < n:
        sys_df = sys_df.append(data.loc[index, :])
        index += sample_count
    return (sys_df)

In [28]:
# df.append 메서드 futurewarning 경고 메시지 출력 무시
import warnings
warnings.filterwarnings(action='ignore')

In [27]:
# 구간 내 필요 샘플 수로 간격 정의
# 함수 호출
sysmetic_sampling(sample_df, 5)

,X_1,X_2,result
2,4,5,2
6,12,13,6
10,20,21,10
14,28,29,14
18,36,37,18


### 층화확률표본추출
- 모집단을 먼저 서로 겹치지 않는 여러 개의 층으로 분할, 각 층별로 단순확률표본추출법을 적용시켜 표본을 추출하는 방법
- Scikit-learn의 StratifiedShuffleSplot(n_splits:분할 반복 횟수, test_size:테스트셋 샘플 비율)
- 각 층의 비율을 고려해 무작위로 train/test set을 분할하는 인덱스를 반환 ( 비례층화추출법에 해당 )

In [30]:
from sklearn.model_selection import StratifiedShuffleSplit

splitfi = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=1001)

In [31]:
sample_df.head(3)

,X_1,X_2,result
0,0,1,0
1,2,3,1
2,4,5,2


In [32]:
# 층을 구분하기 위한 값 ( 0: 0그룹, 1: 1그룹 )
group = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
sample_df['group'] = group

In [33]:
# 데이터 전체 수 확인
sample_df['group'].value_counts()

0    10
1    10
Name: group, dtype: int64

In [34]:
# df_strat_train, df_strat_test 으로 데이터 분할 ( 표본 추출 )
for train_idx, test_idx in splitfi.split(sample_df, sample_df['group']):
    print("Train :", train_idx, "Test :", test_idx)
    df_strat_train = sample_df.loc[train_idx]
    df_strat_test = sample_df.loc[test_idx]

Train : [13  7  1 14 16 12  0 11 10 18  2  8  5  6] Test : [17 19  3 15  4  9]


In [35]:
print("Train data 수 확인")
print(df_strat_train.shape)
print("Test data 수 확인")
print(df_strat_test.shape)

Train data 수 확인
(14, 4)
Test data 수 확인
(6, 4)


In [36]:
# 모집단과 동일 비율로 Group 속성을 기준으로 데이터 분리 확인
print("전체 비율")
print(sample_df['group'].value_counts() / len(sample_df))
print("Train data 비율")
print(df_strat_train['group'].value_counts() / len(df_strat_train))
print("Test data 비율")
print(df_strat_test['group'].value_counts() / len(df_strat_test))

전체 비율
0    0.5
1    0.5
Name: group, dtype: float64
Train data 비율
1    0.5
0    0.5
Name: group, dtype: float64
Test data 비율
1    0.5
0    0.5
Name: group, dtype: float64
